<a href="https://colab.research.google.com/github/masayasato0407/DL_Keras_tabular/blob/main/DL_Keras_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリのインストール
!pip install scikit-optimize keras-tuner

In [ ]:
# 必要なライブラリのインポート
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from kerastuner.tuners import BayesianOptimization

In [ ]:
# training dataのアップロード
uploaded = files.upload()

In [ ]:
# test dataのアップロード
uploaded = files.upload()

In [ ]:
# CSVファイルの読み込み
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
# 説明変数と目的変数 (event)の分割
X_train = train_data.drop('event', axis=1)
y_train = train_data['event']
X_test = test_data.drop('event', axis=1)
y_test = test_data['event']

In [ ]:
# データの標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# モデル構築関数
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# ベイズ最適化によるハイパーパラメータチューニング
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='bayesian_opt'
)

In [ ]:
# モデルの探索
tuner.search(X_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=1)

In [ ]:
# 最良モデルの取得と評価
best_model = tuner.get_best_models(num_models=1)[0]
y_pred_proba = best_model.predict(X_test_scaled).ravel()
y_pred = (y_pred_proba > 0.5).astype(int)

In [ ]:
# 正診率とAUCの計算
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy}')
print(f'ROC AUC: {roc_auc}')

In [ ]:
#Permutation importanceを用いた変数重要度の算出
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

# Kerasモデルのラッパーを作成
class KerasClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y, epochs=10, batch_size=32, verbose=0)
        return self

    def predict(self, X):
        y_pred = (self.model.predict(X) > 0.5).astype("int32")
        return y_pred

    def score(self, X, y):
        # accuracyを使ってスコアを返す
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

# Kerasモデルをラップ
wrapped_model = KerasClassifierWrapper(best_model)

In [ ]:
from sklearn.inspection import permutation_importance

# Kerasモデルのラップしたものを用いて変数重要度を計算
importance = permutation_importance(wrapped_model, X_test_scaled, y_test, n_repeats=10, random_state=0)

# 結果の表示
for i in importance.importances_mean.argsort()[::-1]:
    if importance.importances_mean[i] - 2 * importance.importances_std[i] > 0:
        print(f"{X_train.columns[i]}: {importance.importances_mean[i]:.3f} ± {importance.importances_std[i]:.3f}")

In [ ]:
# 予測結果をCSVに出力
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Probability': y_pred_proba})
output_df.to_csv('prediction_results.csv', index=False)

In [ ]:
# CSVファイルをダウンロードする
files.download('prediction_results.csv')